# set test and train  


In [1]:
import pandas as pd
import numpy as np

In [2]:
rating = pd.read_csv('rating.csv', index_col=0)

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
rating.head()

,movieId,rating,timestamp
userId,,,
1,2,3.5,2005-04-02 23:53:47
1,29,3.5,2005-04-02 23:31:16
1,32,3.5,2005-04-02 23:33:39
1,47,3.5,2005-04-02 23:32:07
1,50,3.5,2005-04-02 23:29:40


In [4]:
flattened = pd.DataFrame(rating.to_records())

In [5]:
flattened.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
sample = flattened.head(100000)

In [11]:
user_movie_table = pd.pivot_table(sample, index=['userId'], columns=['movieId'])

# Going to make this as a CLASS!

In [14]:
from random import shuffle

In [15]:
def make_train_test(user_movie_table, user_num):
    mu = user_movie_table.T
    user1 = mu[user_num]
    
    watched_index = []
    unseen_index = []
    for i in range(len(user1)):
        if user1[i] >=0:
            watched_index.append(i)
        else:
            unseen_index.append(i)
    shuffle(watched_index)
    test_index = watched_index[:round(0.1*len(watched_index))]
    train_index_temp = list(map(lambda x: x if x not in test_index else np.nan, watched_index))
    train_index = list(pd.Series(train_index_temp).dropna().apply(int))
    
    return train_index, test_index, watched_index, unseen_index
    

In [16]:
train, test, watched_index, unseen_index = make_train_test(user_movie_table, 1)

In [17]:
def weight_calculator(user_movie_table, index1, index2):
    """find pearson corr between two users
    Parameters
    --------------
    user_movie_table : pd.DataFrame
        user_movie rating info
    index1, index2 : Integer
        pointer in table
    """
    mu = user_movie_table.T
    user1 = mu[index1]
    user2 = mu[index2]
    common = mu[[index1,index2]].dropna()
    user1_watched = user1.dropna()
    user2_watched = user2.dropna()
    
    x1_mean = user1.mean()
    x2_mean = user2.mean()
    numerator = np.array(list(map(lambda x,y : (x-x1_mean)*(x-x2_mean), common[index1],common[index2]))).sum()
    
    left_corner = np.array(list(map(lambda x : (x-x1_mean)**2, user1_watched))).sum()**0.5
    right_corner = np.array(list(map(lambda x : (x-x2_mean)**2, user2_watched))).sum()**0.5
    denominator = left_corner * right_corner
    
    weight_between_two_users = numerator/denominator
    
    return weight_between_two_users

In [12]:
# w = weight_calculator(user_movie_table, 1, 2)

### => Note! I found the weight for a person to himself is not 1!

# Now, take the user1 as the target. Let's say I want to know whether I should recommend movie1 to him.

user3,6,8,10 has review for movie1

In [18]:
def predict_ur_score(user_movie_table, user_num, movie_index):
    
    mu = user_movie_table.T

    #1.find people who rates movie3
    movie3_rating = user_movie_table.ix[:,movie_index]
    movie3_rating = movie3_rating.dropna() #so I find the movie3 all user ratings
    
    #2.get users who have rated movie3
    movieId = pd.DataFrame(user_movie_table.ix[user_num,:]).reset_index()['movieId']
    users_movie3 = list(pd.DataFrame(movie3_rating.reset_index())['userId'])
    movie3_rating_score = list(pd.DataFrame(movie3_rating.reset_index())['rating'][movieId[movie_index]])
    
    #3.calculate predict score for user1 of movie3
    numerator_list = []
    denominator_list = []
    for i in range(len(users_movie3)):
        weight_1_i = weight_calculator(user_movie_table, user_num, users_movie3[i])

        useri_watched = mu[users_movie3[i]].dropna()
        xi_mean = useri_watched.mean()

        score_i = movie3_rating_score[i]
        numerator_i = weight_1_i * (score_i - xi_mean)
        numerator_list.append(numerator_i)

        denominator_i = (weight_1_i**2)**0.5
        denominator_list.append(denominator_i)
    
    numerator = np.array(numerator_list).sum()
    denominator = np.array(denominator_list).sum()
    
    user1_watched = mu[user_num].dropna()
    x1_mean = user1_watched.mean()
    socre_1_3 = x1_mean + numerator/denominator
    
    return socre_1_3

In [17]:
# socre_1_3 = predict_ur_score(user_movie_table, 1, 2)

# NOW, predict all score for those unseen movies for user1

In [19]:
import operator

In [40]:
def pred_score_for_user_i(user_movie_table, user_num, unseen_index):
    
    movie_pred_score = {}
    for i in range(len(unseen_index)):
        score_i = predict_ur_score(user_movie_table, 1, unseen_index[i])
        movie_pred_score[unseen_index[i]] = score_i
        sorted_movie_pred_score = sorted(movie_pred_score.items(), key=operator.itemgetter(1), reverse=True)
    
    recommend_movie_index = list(map(lambda tp: tp[0],sorted_movie_pred_score))
    recommend_movie_score = list(map(lambda tp: tp[1],sorted_movie_pred_score))
    
    movieId = pd.DataFrame(user_movie_table.ix[user_num,:]).reset_index()['movieId']
    recommend_movieId = [movieId[i] for i in recommend_movie_index]
    
    return recommend_movieId, recommend_movie_score

In [33]:
index, score = pred_score_for_user_i(user_movie_table, 1, test )

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-i

In [41]:
pred = pd.DataFrame()
pred['movieId'] = index
pred['pred_score'] = score
pred

,movieId,pred_score
0,8507,4.747154
1,47,4.211586
2,1196,4.186652
3,260,4.169932
4,924,4.132900
5,6093,4.039545
6,1214,4.033746
7,7757,4.020403
8,1097,3.950299
9,2118,3.948383


In [42]:
#true score
user_movie_table.ix[1, test]

C:\Users\taliu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


        movieId
rating  5999       3.5
        2118       4.0
        47         3.5
        1348       3.5
        924        3.5
        4720       3.5
        5040       3.0
        7757       4.0
        5898       3.5
        1214       4.0
        1097       4.0
        260        4.0
        6093       4.0
        2          3.5
        1196       4.5
        8507       5.0
        2174       4.0
        3476       3.5
Name: 1, dtype: float64

# ?? Why there is one movie's rating is more than 5????